In [39]:
import xarray as xr
import os
import random
import subprocess
from datetime import datetime

# ----------------------- USER SPECIFIES --------------------------
DATE = "2025-08-08"  # format: YYYY-MM-DD
ensembleSize = 11
CAMIC_TXT_DIR = "/glade/work/sglanvil/CCR/kjmayer/camic_txt_files/"
FINAL_IC_DIR = "/glade/derecho/scratch/kjmayer/CREDIT_runs/S2Socnlndatm_MLdata/AIWQ_inits/"
# -----------------------------------------------------------------
os.makedirs(CAMIC_TXT_DIR, exist_ok=True)
date_obj = datetime.strptime(DATE, "%Y-%m-%d")
YEAR = date_obj.year
MONTH = date_obj.month
DAY = date_obj.day


counter=-11
GEFSmbrs = ['c00'] + [f'p{str(i).zfill(2)}' for i in range(1, 31)]
for GEFSmbr in GEFSmbrs:
    print()
    original_file = f'/glade/derecho/scratch/dgagne/GEFS_CAM/gefs.20250808/00/gefs_cam_grid_{GEFSmbr}.nc'
    print(f'GEFS mbr: {GEFSmbr} -- GEFS file: {original_file}')
    # ------------------- CREATE LIST OF PERTURBATION FILE VALUES ----------------------
    numbers = []
    seen = set()
    while len(numbers) < ensembleSize // 2:
        rand_num = random.randint(1, 999)
        if rand_num not in seen:
            numbers.append(rand_num)
            seen.add(rand_num)
    filename = os.path.join(CAMIC_TXT_DIR, f"camic_{DATE}.1-10_GEFS{GEFSmbr}.txt")
    with open(filename, "w") as f:
        f.write(" ".join(map(str, numbers)))
    with open(filename, "r") as f:
        print('random perts:',f.read())
    # --------------------------- MAKE THE ENSEMBLE MEMBERS ---------------------------
    counter=counter+11
    for i in range(1, ensembleSize + 1):
        print()
        mbr = f"{i:03d}"
        mbr_counter = i + counter
        mbr_forFilename = f"{mbr_counter:03d}"
        final_file = os.path.join(FINAL_IC_DIR, f"GEFS_ICs.{mbr_forFilename}.i.{DATE}-00000.nc")
        print(f"final: {final_file}")
        if mbr == "001":
            print(f"mbr: 001, use original: {original_file}")
            subprocess.run(["cp", original_file, final_file], check=True)
        else:
            mbr_num = int(mbr)
            if mbr_num % 2 == 0:
                inx = (mbr_num - 2) // 2
                weight = 0.15
            else:
                inx = (mbr_num - 3) // 2
                weight = -0.15
            pert_number = numbers[inx]
            pert_file = f"/glade/campaign/cesm/development/cross-wg/S2S/CESM2/CAMI/RP/{MONTH:02d}/CESM2.cam.i.M{MONTH:02d}.diff.{pert_number}.nc"
            pert_file_tmp = "/glade/work/sglanvil/CCR/kjmayer/pert_file_tmp.nc"
            ds = xr.open_dataset(pert_file)
            ds = ds.rename({'US': 'U', 'VS': 'V'})
            ds['U'] = ds['U'].interp(slat=ds['lat'], lon=ds['lon'], method='nearest')
            ds['V'] = ds['V'].interp(slon=ds['lon'], lat=ds['lat'], method='nearest')
            ds = ds.rename({'Q': 'Qtot', 'lev': 'level', 'lat': 'latitude', 'lon': 'longitude'})
            ds.to_netcdf(pert_file_tmp)
            print(f"mbr: {mbr}, inx: {inx}, weight: {weight}, use pert: {pert_file}")
            # cmd = f'module load nco && ncflint -O -C -v lat,lon,slat,slon,lev,ilev,hyai,hybi,hyam,hybm,U,V,T,Q,PS -w {weight},1.0 {pert_file_tmp} {original_file} {final_file}'
            cmd = f'module load nco && ncflint -O -C -v latitude,longitude,level,U,V,T,Qtot,PS -w {weight},1.0 {pert_file_tmp} {original_file} {final_file}'
            os.system(f'bash -lc "{cmd} > /dev/null 2>&1"')
            ds_interp = xr.open_dataset(final_file).load()
            ds_interp = ds_interp.drop_vars(['latitude', 'longitude', 'level'], errors='ignore')
            ds_orig = xr.open_dataset(original_file).load()
            ds_final = xr.merge([ds_interp, ds_orig.drop_vars(ds_interp.data_vars.keys())])
            os.remove(final_file)
            ds_final.to_netcdf(final_file)




GEFS mbr: c00 -- GEFS file: /glade/derecho/scratch/dgagne/GEFS_CAM/gefs.20250808/00/gefs_cam_grid_c00.nc
random perts: 533 55 545 864 874
final: /glade/derecho/scratch/kjmayer/CREDIT_runs/S2Socnlndatm_MLdata/AIWQ_inits/GEFS_ICs.001.i.2025-08-08-00000.nc
mbr: 001, use original file
/glade/derecho/scratch/dgagne/GEFS_CAM/gefs.20250808/00/gefs_cam_grid_c00.nc
final: /glade/derecho/scratch/kjmayer/CREDIT_runs/S2Socnlndatm_MLdata/AIWQ_inits/GEFS_ICs.002.i.2025-08-08-00000.nc
mbr: 002, inx: 0, weight: 0.15
/glade/campaign/cesm/development/cross-wg/S2S/CESM2/CAMI/RP/08/CESM2.cam.i.M08.diff.533.nc
final: /glade/derecho/scratch/kjmayer/CREDIT_runs/S2Socnlndatm_MLdata/AIWQ_inits/GEFS_ICs.003.i.2025-08-08-00000.nc
mbr: 003, inx: 0, weight: -0.15
/glade/campaign/cesm/development/cross-wg/S2S/CESM2/CAMI/RP/08/CESM2.cam.i.M08.diff.533.nc
final: /glade/derecho/scratch/kjmayer/CREDIT_runs/S2Socnlndatm_MLdata/AIWQ_inits/GEFS_ICs.004.i.2025-08-08-00000.nc
mbr: 004, inx: 1, weight: 0.15
/glade/campaign/